# Data collection hourly
Gather public SPP Weis data from https://marketplace.spp.org/groups/operational-data-weis

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os
import sys

# logging
import logging

# define log
logging.basicConfig(level=logging.INFO)

# Set the logging level for the 'py4j' logger to ERROR or WARNING
logging.getLogger("py4j").setLevel(logging.ERROR) 

log = logging.getLogger(__name__)


In [0]:
# set env vars
if dbutils:
    os.environ['AWS_ACCESS_KEY_ID'] = dbutils.secrets.get(scope = "aws", key = "AWS_ACCESS_KEY_ID")
    os.environ['AWS_SECRET_ACCESS_KEY'] = dbutils.secrets.get(scope = "aws", key = "AWS_SECRET_ACCESS_KEY")

from dotenv import load_dotenv
load_dotenv()

In [0]:
os.chdir('../..')
sys.path.append('./src')

In [0]:
import src.data_collection as dc

## Mid Term Load Forecast

![_](../../imgs/mtlf.PNG)

In [0]:
dc.collect_upsert_mtlf(n_periods=24)

## Mid Term Resource Forecast

![_](../../imgs/mtrf.PNG)

In [0]:
dc.collect_upsert_mtrf(n_periods=24)